In [ ]:
# Model Training:
# 1) Load all data from preprocessing (training/test splits, etc)
# 2) Begin Training Models
    #  a) Decision Tree
    #  b) Naive Bayes
    #  c) Logistic Regression
    #  d) SVM
# 3) Testing Models
# 4) New Iterations

In [14]:
# Imports
from Models import ModelUtil
from Data import Preprocessing
from Visualization import VisualUtil
from Logs import logging as logs

import importlib
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import LeaveOneOut
import configparser
import numpy as np

config = configparser.ConfigParser()
config.read('Data//config.ini')

importlib.reload(Preprocessing)
importlib.reload(ModelUtil)
importlib.reload(VisualUtil)
importlib.reload(logs)

<module 'Logs.logging' from 'c:\\Users\\Trent\\Desktop\\Senior Design\\shifting_model\\Logs\\logging.py'>

In [15]:
# 1) Load all data from preprocessing 
importlib.reload(Preprocessing)

if("False" in config['DATA']['USE_NEW_PREPROCESSING']):
    infieldDataFrame, outfieldDataFrame = Preprocessing.dataFiltering([], False)
else:
    normDataFrame = Preprocessing.dataProcessing()
    infieldDataFrame, outfieldDataFrame = Preprocessing.dataFiltering(normDataFrame, True)

c:\Users\Trent\Desktop\Senior Design\shifting_model\Data\DataUtil.py:302: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["PitcherThrows"] = df["PitcherThrows"].map({"Left":1, "Right":2, "Both":3})


In [8]:
infieldcorrmatrix = infieldDataFrame.corr()
infieldcorrmatrix.to_csv('Infield_Correlation_Matrix')
outfieldcorrmatrix = outfieldDataFrame.corr()
outfieldcorrmatrix.to_csv('Outfield_Correlation_Matrix')

ValueError: could not convert string to float: ''

In [17]:
importlib.reload(Preprocessing)
# 2) Training Models

if("False" in config['DATA']['USE_NEW_PREPROCESSING']):
    Y = infieldDataFrame["FieldSlice"]
    X = infieldDataFrame[["PitcherThrows", "BatterSide", "TaggedPitchType", "ZoneSpeed", "PlateLocHeight", "PlateLocSide"]]
    xTrain, xTest, yTrain, yTest = train_test_split(X, Y, test_size=0.25, random_state=11)
else:
    infieldY = infieldDataFrame[0][['Direction','Distance']]
    infieldX = infieldDataFrame[0][infieldDataFrame[1]] 
    if("True" in config['SPLIT']['TTS']):
        xTrain, xTest, yTrain, yTest = train_test_split(infieldX, infieldY, test_size=0.20, random_state=11)
        
    elif("True" in config['SPLIT']['KFold']):
        kf = KFold(n_splits=5, shuffle=True)
        for train_index, test_index in kf.split(infieldX):
            xTrain, xTest = infieldX.iloc[train_index,:], infieldX.iloc[test_index,:]
            yTrain, yTest = infieldY.iloc[train_index,:], infieldY.iloc[test_index,:]

    elif("True" in config['SPLIT']['LOOCV']):
        loo = LeaveOneOut()
        for train_index, test_index in loo.split(infieldX):
            xTrain, xTest = infieldX.iloc[train_index,:], infieldX.iloc[test_index,:]
            yTrain, yTest = infieldY.iloc[train_index,:], infieldY.iloc[test_index,:]

    else:
        print("No Splitting Method Selected")
        

# GroupKFold: (avoids putting data from the same group in the test set -- useful for Pitcher/Batter ID when we implement that.)

In [23]:
importlib.reload(ModelUtil)
importlib.reload(logs)
# a) Decision Tree
# Need to test these hyperparameters for best case
max_depth = 25
max_features = 10
max_leaf_nodes = 55
result = ModelUtil.runDT(xTrain, yTrain, xTest, yTest, max_depth, max_features, max_leaf_nodes)


training decision tree model...
done!
getting statistics...

printing statistics...
Model Type: DecisionTree

Training Size = 42213
Testing Size = 14072

Training Accuracy = 0.32906924407173144
Testing Accuracy = 0.31807845366685616

Hyper-Parameters: 

Max Tree Depth: 25
Max Tree Features: 10
Max Leaf Nodes: 55

Accuracy Score for Predicting on Training Data: 1.0890
Accuracy Score for Predicting on Test Data: 0.3181

Overall Average Probabilities
-------------------------------------
Section 1: 20.73%
Section 2: 20.66%
Section 3: 20.24%
Section 4: 19.59%
Section 5: 18.77%

Field Slice Counts for Training Data
--------------------------------------------------
Section	Truth	Prediction
1		7295		14408
2		10967		9860
3		10348		3422
4		8483		7320
5		5120		7203
Amount Correct: 13891
Amount Incorrect: 28322

Field Slice Counts for Testing Data
--------------------------------------------------
Section	Truth	Prediction
1		2513		4906
2		3643		3219
3		3427		1155
4		2881		2431
5		1608		2361
Amou

c:\Users\Trent\Desktop\Senior Design\shifting_model\Logs\logging.py:153: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  dfTestStats = dftest.groupby(["FieldSliceActual"]).size().reset_index()
c:\Users\Trent\Desktop\Senior Design\shifting_model\Logs\logging.py:157: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  dfTestStats["Correct"] = dftemp.groupby(["FieldSliceActual"]).size().reset_index()[0]
c:\Users\Trent\Desktop\Senior Design\shifting_model\Logs\logging.py:159: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain

In [24]:
importlib.reload(ModelUtil)
importlib.reload(logs)
# b) Naive Bayes

var_smoothing = 1e-9
result = ModelUtil.runNB(xTrain, yTrain, xTest, yTest, var_smoothing)

training Naive Bayes model...
done!
getting statistics...
printing statistics...
Model Type: NaiveBayes

Training Size = 42213
Testing Size = 14072

Training Accuracy = 0.31431075734963165
Testing Accuracy = 0.31772313814667424

Hyper-Parameters: 

Var Smoothing: 1e-09

Accuracy Score for Predicting on Training Data: 1.0389
Accuracy Score for Predicting on Test Data: 0.3177

Overall Average Probabilities
-------------------------------------
Section 1: 19.20%
Section 2: 24.67%
Section 3: 23.61%
Section 4: 20.37%
Section 5: 12.15%

Field Slice Counts for Training Data
--------------------------------------------------
Section	Truth	Prediction
1		7295		7620
2		10967		19899
3		10348		497
4		8483		11310
5		5120		2887
Amount Correct: 13268
Amount Incorrect: 28945

Field Slice Counts for Testing Data
--------------------------------------------------
Section	Truth	Prediction
1		2513		2603
2		3643		6627
3		3427		139
4		2881		3790
5		1608		913
Amount Correct: 4471
Amount Incorrect: 9601
done!


c:\Users\Trent\Desktop\Senior Design\shifting_model\Logs\logging.py:153: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  dfTestStats = dftest.groupby(["FieldSliceActual"]).size().reset_index()
c:\Users\Trent\Desktop\Senior Design\shifting_model\Logs\logging.py:157: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  dfTestStats["Correct"] = dftemp.groupby(["FieldSliceActual"]).size().reset_index()[0]
c:\Users\Trent\Desktop\Senior Design\shifting_model\Logs\logging.py:159: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain

In [25]:
importlib.reload(ModelUtil)
importlib.reload(logs)
# c)Logistic Regression
lr = 0.8
e = 100
result = ModelUtil.runLogReg(xTrain, yTrain, xTest, yTest, lr, e)

training logistic regression model...
done!
getting statistics...
printing statistics...
Model Type: LogisticRegression

Training Size = 42213
Testing Size = 14072

Training Accuracy = 0.3006182929429323
Testing Accuracy = 0.30144968732234223

Hyper-Parameters: 

Learning Rate: 0.8
Epochs: 100

Accuracy Score for Predicting on Training Data: 1.1878
Accuracy Score for Predicting on Test Data: 0.3014

Overall Average Probabilities
-------------------------------------
Section 1: 20.69%
Section 2: 20.59%
Section 3: 20.07%
Section 4: 19.53%
Section 5: 19.12%

Field Slice Counts for Training Data
--------------------------------------------------
Section	Truth	Prediction
1		7295		14980
2		10967		9536
3		10348		2854
4		8483		3485
5		5120		11358
Amount Correct: 12690
Amount Incorrect: 29523

Field Slice Counts for Testing Data
--------------------------------------------------
Section	Truth	Prediction
1		2513		5125
2		3643		3164
3		3427		906
4		2881		1102
5		1608		3775
Amount Correct: 4242
Am

C:\Users\Trent\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\Users\Trent\Desktop\Senior Design\shifting_model\Logs\logging.py:153: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  dfTestStats = dftest.groupby(["FieldSliceActual"]).size().reset_index()
c:\User

In [26]:
importlib.reload(ModelUtil)
importlib.reload(logs)
# d) SVM
rC = 1
kernel='linear'
degree= 1
gamma= 'scale'
coef0= 0.0
result = ModelUtil.runSVM(xTrain, yTrain, xTest, yTest, rC, kernel, degree, gamma, coef0)

training SVM model...


In [ ]:
# z) RandomForestRegressor
for i in range(0, len(trainIn)):
    direction, distance = ModelUtil.runRFR(trainIn[i], trainOut[i], testIn[i], testOut[i])

In [ ]:
# 3) Model Testing:

In [ ]:
# 4) Model Iterations and Improvements

In [6]:
# 5) Data Visualization

# Temporary method of getting percentages for testing purposes
slices = 5
infieldPercentages  = np.random.dirichlet(np.ones(slices), size=1)[0]
outfieldPercentages = np.random.dirichlet(np.ones(slices), size=1)[0]

VisualUtil.visualizeData(infieldPercentages, outfieldPercentages)


(12, 35, 64)
#0c2340
